In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logomaker

pd.options.display.max_colwidth = 500

def _plot_weights(array, path, figsize=(10, 3)):
    """Plot weights as a sequence logo and save to file."""
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)

    df = pd.DataFrame(array, columns=['A', 'C', 'G', 'T'])
    df.index.name = 'pos'

    crp_logo = logomaker.Logo(df, ax=ax)
    crp_logo.style_spines(visible=False)
    plt.ylim(min(df.sum(axis=1).min(), 0), df.sum(axis=1).max())

    plt.savefig(path)
    plt.close()

def create_seqlet_logos(seqlets, output_dir, pattern_group):
    """Create logos for each seqlet cluster and save them to the output directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    for i, seqlet in enumerate(seqlets):
        # Compute information content (IC)
        ic = np.sum(seqlet * np.log2(seqlet + 1e-6), axis=1)  # Pseudocount to avoid log(0)

        # Generate the logo
        logo_path = os.path.join(output_dir, f'{pattern_group}_seqlet_{i}.png')
        _plot_weights(seqlet * ic[:, None], path=logo_path)

def path_to_image_html(path):
    return f'<img src="{path}" width="240">'

def generate_report(attributions_file, seqlets_file, clusters_file, output_dir, img_path_suffix):
    # Load attributions, seqlets, and clustering labels
    if not os.path.exists(attributions_file):
        print(f"File not found: {attributions_file}")
        return
    if not os.path.exists(seqlets_file):
        print(f"File not found: {seqlets_file}")
        return
    if not os.path.exists(clusters_file):
        print(f"File not found: {clusters_file}")
        return

    attributions = np.load(attributions_file)
    seqlets = np.load(seqlets_file)
    clusters = np.load(clusters_file)
    
    # Create output directories
    logos_dir = os.path.join(output_dir, 'logos')
    os.makedirs(logos_dir, exist_ok=True)
    
    # Create logos for each cluster
    unique_clusters = set(clusters)
    pattern_groups = ['cluster_' + str(c) for c in unique_clusters if c != -1]

    for cluster in unique_clusters:
        if cluster == -1:
            continue  # Skip noise points
        cluster_seqlets = seqlets[clusters == cluster]
        create_seqlet_logos(cluster_seqlets, logos_dir, f'cluster_{cluster}')

    # Generate report dataframe
    results = {'cluster': [], 'num_seqlets': [], 'logo_path': []}
    
    for cluster in unique_clusters:
        if cluster == -1:
            continue  # Skip noise points
        num_seqlets = np.sum(clusters == cluster)
        logo_path = os.path.join('logos', f'cluster_{cluster}_seqlet_0.png')
        
        results['cluster'].append(f'Cluster {cluster}')
        results['num_seqlets'].append(num_seqlets)
        results['logo_path'].append(logo_path)
    
    report_df = pd.DataFrame(results)
    
    # Create HTML report
    html_path = os.path.join(output_dir, 'motif_report.html')
    report_df.to_html(html_path, escape=False, formatters={'logo_path': path_to_image_html}, index=False)
    print(f"Motif report generated successfully at {html_path}")

# Use the current working directory if __file__ is not defined
if '__file__' in globals():
    base_dir = os.path.dirname(os.path.abspath(__file__))
else:
    base_dir = os.getcwd()

# Paths to required files
attributions_file = os.path.join(base_dir, 'results/attributions/attributions_subset.npy')
seqlets_file = os.path.join(base_dir, 'results/seqlets/high_attribution_seqlets_subset.npy')
clusters_file = os.path.join(base_dir, 'results/clusters/seqlet_clusters_subset.npy')
output_dir = os.path.join(base_dir, 'results/report')
img_path_suffix = 'logos'  # Set relative path for HTML image links

# Ensure the output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, img_path_suffix), exist_ok=True)

# Generate motif report
generate_report(attributions_file, seqlets_file, clusters_file, output_dir, img_path_suffix)

print("Motif report generation complete.")


Motif report generated successfully at /Users/ciarareeve/motify_new/results/report/motif_report.html
Motif report generation complete.


In [6]:
!pwd


/Users/ciarareeve/motify_new
